## Reading large QTl files based off QTLStudy structure

In [1]:
using DLMReader
using Chain
using InMemoryDatasets
import Base.Threads.@threads
using StatsBase

export QTLStudy, GWAS, QTLStudy_from_pattern, GenVarInfo
include("../src/inputs.jl")
include("../src/mrStudyCis.jl")

mrStudyCis

In [2]:
GWAS_file_path = "/mnt/r6_data/GWAS_MRsuite_hg38/input_MRsuite_BMI_Yengo_hg38"
QTL_prots_path = "/home/matpa01/CrickData_matpa01/Decode_mapping_suite"
QTL_prots_tss_file = "/mnt/r6_data/Patrick/Input_Decode/decode_list_hg38"

tss_df = filereader(QTL_prots_tss_file, delimiter = '_', header=[:trait, :chr, :tss], eolwarn = false)

dropmissing!(tss_df)

Decode_pat = [TRAIT_NAME, "/", TRAIT_NAME, "_chromosome_", CHR, ".tsv"]
Decode_columns = Dict(1 => "id", 2 => CHR, 3 => POS, 4 => A_EFFECT, 5 => A_OTHER, 6 => BETA, 7 => SE, 8 => PVAL)
sep = '\t'

Decode = QTLStudy_from_pattern(QTL_prots_path,Decode_pat, tss_df.trait, tss_df.chr, tss_df.tss, Decode_columns, sep, true)

display(Decode.path_v)
display(Decode.traits_for_each_path)

┌ Warning: There are problems with parsing the input file at line 723 (observation 723) : 
│ Column 2 : chr::Int64 : Read from buffer ("Y")
│  the values are set as missing.
│ MORE DETAILS: 
│ trait::String = CDY1.7097.8, chr::Int64 = missing, tss::Int64 = 25622162
│ CDY1.7097.8_Y_25622162
└ @ DLMReader ~/.julia/packages/DLMReader/N5BLJ/src/util.jl:952
┌ Warning: There are problems with parsing the input file at line 1295 (observation 1295) : 
│ Column 2 : chr::Int64 : Read from buffer ("Y")
│  the values are set as missing.
│ MORE DETAILS: 
│ trait::String = EIF1AY.19224.5, chr::Int64 = missing, tss::Int64 = 20575776
│ EIF1AY.19224.5_Y_20575776
└ @ DLMReader ~/.julia/packages/DLMReader/N5BLJ/src/util.jl:952


4483-element Vector{String}:
 "/home/matpa01/CrickData_matpa01" ⋯ 34 bytes ⋯ "/A1BG.16561.9_chromosome_19.tsv"
 "/home/matpa01/CrickData_matpa01" ⋯ 36 bytes ⋯ "A1CF.12423.38_chromosome_10.tsv"
 "/home/matpa01/CrickData_matpa01" ⋯ 32 bytes ⋯ "2/A2M.3708.62_chromosome_12.tsv"
 "/home/matpa01/CrickData_matpa01" ⋯ 38 bytes ⋯ "2ML1.15482.12_chromosome_12.tsv"
 "/home/matpa01/CrickData_matpa01" ⋯ 38 bytes ⋯ "4GALT.8759.29_chromosome_22.tsv"
 "/home/matpa01/CrickData_matpa01" ⋯ 37 bytes ⋯ "A4GNT.10835.25_chromosome_3.tsv"
 "/home/matpa01/CrickData_matpa01" ⋯ 38 bytes ⋯ "AGAB.11926.23_chromosome_15.tsv"
 "/home/matpa01/CrickData_matpa01" ⋯ 44 bytes ⋯ "HPPT.17320.19_chromosome_11.tsv"
 "/home/matpa01/CrickData_matpa01" ⋯ 36 bytes ⋯ "ABCC6.8935.22_chromosome_16.tsv"
 "/home/matpa01/CrickData_matpa01" ⋯ 41 bytes ⋯ "HD10.17370.186_chromosome_3.tsv"
 "/home/matpa01/CrickData_matpa01" ⋯ 36 bytes ⋯ "ABHD12.7825.7_chromosome_20.tsv"
 "/home/matpa01/CrickData_matpa01" ⋯ 41 bytes ⋯ "HD14B.19325.21_chrom

4483-element Vector{Any}:
 "A1BG.16561.9"
 "A1CF.12423.38"
 "A2M.3708.62"
 "A2ML1.15482.12"
 "A4GALT.8759.29"
 "A4GNT.10835.25"
 "AAGAB.11926.23"
 "AASDHPPT.17320.19"
 "ABCC6.8935.22"
 "ABHD10.17370.186"
 "ABHD12.7825.7"
 "ABHD14B.19325.21"
 "ABL1.3341.33"
 ⋮
 "ZNF566.12795.2"
 "ZNF774.12760.34"
 "ZNF843.8321.27"
 "ZNRF3.10390.21"
 "ZNRF3.14122.132"
 "ZP4.7766.25"
 "ZPBP.8635.283"
 "ZPBP2.10710.23"
 "ZPLD1.5590.11"
 "ZWILCH.18302.204"
 "ZWINT.18286.3"
 "ZYX.13632.10"

Read each file : keep only relevant cols and smoosh it all together

In [28]:
Decode[1]

GWAS("/home/matpa01/CrickData_matpa01/Decode_mapping_suite/A1BG.16561.9/A1BG.16561.9_chromosome_19.tsv", Dict{Int64, GenVarInfo}(5 => A_OTHER, 4 => A_EFFECT, 6 => BETA, 7 => SE, 2 => CHR, 8 => PVAL, 3 => POS), '\t', "A1BG.16561.9")

In [34]:
verify_and_simplify_columns(Decode)

-1

In [35]:
Decode.columns

Dict{Int64, GenVarInfo} with 7 entries:
  5 => A_OTHER
  4 => A_EFFECT
  6 => BETA
  7 => SE
  2 => CHR
  8 => PVAL
  3 => POS

In [36]:
types, header = make_types_and_headers(Decode)

n_cols_file = 8


(Dict{Int64, DataType}(5 => String, 4 => String, 6 => Float64, 7 => Float64, 2 => Int8, 8 => Float64, 3 => Int32, 1 => String), [:x, :chr, :pos, :a_effect, :a_other, :β, :se, :pval])

In [37]:
types

Dict{Int64, DataType} with 8 entries:
  5 => String
  4 => String
  6 => Float64
  7 => Float64
  2 => Int8
  8 => Float64
  3 => Int32
  1 => String

In [38]:
header

8-element Vector{Symbol}:
 :x
 :chr
 :pos
 :a_effect
 :a_other
 :β
 :se
 :pval

In [3]:
df = mrStudyCis(Decode, Decode[1])